In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from preprocessor import Processor

In [3]:
util = Util()
cleaner = DataCleaner()
processor = Processor()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\natnael.masresha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up

In [4]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
news_df=util.read_from_dvc("data/news.csv",repo,"news-v0",low_memory=False)
news_df=news_df.sample(frac=1)

# cleaned_df=util.read_from_file('../data/news.csv')

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=710796635688-iivsgbgsb6uv1fap6635dhvuei09o66c.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.appdata&access_type=offline&response_type=code&approval_prompt=force

Authentication successful.


In [5]:
train_news=news_df.head(8)
test_news=news_df.tail(2)

In [6]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [7]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)


In [8]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [9]:
full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

,Domain,Title,Description,Body,timestamp,Analyst Average Score
0,mype za,male arrest murder elderli femal cofimvaba sap...,crime stamp n1 r101 road appear court sap crim...,south african polic servic offic provinci comm...,2021 09 10t00 17 46 055622,1.33
1,news24 com,news24 com court dismiss attempt eskom ceo pun...,lawsuit public particip design intimid johanne...,eskom ceo matshela mose koko seek r500 000 dam...,2021 09 09t19 32 46 239682,0.33
2,eminetra za,construct sector expect boost riot loot repair,addit spend build repair secur kwazulu natal h...,construct activ grow steadili 4 second quarter...,2021 09 09t09 02 46 320793,1.66
3,atp tchad info,marché résine dan le peintur et revêtement 202...,covid 19 covid covid covid 19 et post covid co...,le rapport d étude marché résine dan le peintu...,2021 09 13t07 32 46 244403,0.00
4,manometcurr com,global region beta caroten market research 202...,key player dsm basf alli biotech chr histor ma...,global region beta caroten market research 202...,2021 09 13t03 02 45 609228,0.00


In [10]:

processor.prepare_tuner(full_promp_trainer)

tuner prepared successfuly


In [11]:
# uncomment these for first time use
# full_processed_df.to_csv("../data/trainer_news.csv",index=False)
# processed_test.to_csv("../data/test_news.csv",index=False)